## Data Acquisition
In this notebook we will load some earth observation data from the Sentinel-2
satellite and show some basic operations on the data. This includes visualizing
the data, calculating the NDVI and exporting the data to a GeoTIFF file.

In [ ]:
from datetime import datetime, timedelta
from pathlib import Path   # noqa

import xarray as xr   # noqa
import pystac_client
import odc.stac
import rioxarray  # noqa
from odc.geo.geobox import GeoBox

import matplotlib.pyplot as plt

Before we start, we need to load the data. We will use ``odc-stac`` to obtain
data from Earth Search by Element 84. Here we define the area of interest and
the time frame, aswell as the EPSG code and the resolution.

### Searching in the Catalog
The module ``odc-stac`` provides access to free, open source satelite data.
To retrieve the data, we must define  several parameters that specify the
location and time period for the satellite data. Additionally, we must specify
the data collection we wish to access, as multiple collections are available.
In this example, we will use multispectral imagery from the Sentinel-2
satellite.

In [ ]:
# Set up the geometry
dx = 0.0006  # 60m resolution
epsg = 4326

# Set Spatial extent
latmin, latmax = 47.8, 48
lonmin, lonmax = 16.7, 16.9
bounds = (lonmin, latmin, lonmax, latmax)

# Set Temporal extent
start_date = datetime(year=2024, month=5, day=1)
end_date = start_date + timedelta(days=14)
date_query = (
    start_date.strftime("%Y-%m-%d") + "/" + end_date.strftime("%Y-%m-%d")
)

# Search for Sentinel-2 data
items = (
    pystac_client.Client.open("https://earth-search.aws.element84.com/v1")
    .search(
        bbox=bounds,
        collections=["sentinel-2-l2a"],
        datetime=date_query,
        limit=100,
    )
    .item_collection()
)
print(len(items), "scenes found")

We will now focus on the area south-east of Vienna, where the Nationalpark 
_Donauauen_ is situated. The time frame we are interested in is the beginning
of May 2024. After passing these parameters to the `stac-catalog` we have found
some scenes that we can use for our analysis. 

### Loading the Data
Now we will load the data directly into an ``xarray`` dataset, which we can use
to perform computations on the data. ``xarray`` is a powerful library for
working with multi-dimensional arrays, making it well-suited for handling
satellite data.

First to get a grasp of what we can actually load from the catalog, we can 
inspect the items and find out what bands are available. We can find the bands 
in the assets of an item.

In [ ]:
# Print the bands available 
# (excluding the Jpeg2000 files, but they could be used aswell)
for band in items[0].assets:
    if "-jp2" not in band:
        print(band)

# Inspect the first item interactively in the notebook
items[0]

Here's how we can load the data using odc-stac and xarray:

In [ ]:
# define a geobox for my region
geobox = GeoBox.from_bbox(bounds, crs=f"epsg:{epsg}", resolution=dx)

# lazily combine items into a datacube
dc = odc.stac.load(
    items,
    bands=["scl", "red", "green", "blue", "nir"],
    chunks={'time': 5, 'x': 600, 'y': 600},
    geobox=geobox,
    resampling="bilinear",
)
dc

## Data Visualization
### RGB Image
With the image data now in our possession, we can proceed with computations
and visualizations.

First, we define a mask to exclude cloud cover and areas with missing data.
Subsequently, we create a composite median image, where each pixel value
represents the median value across all the scenes we have identified. This
approach helps to eliminate clouds and outliers present in some of the images,
thereby providing a clearer and more representative visualization of the scene.

Keep in mind, that the color vector in each pixel is:
$$
\vec{c} = \begin{bmatrix} Red \\ Green \\ Blue \end{bmatrix}
$$
This is evident in the following code snippet, where we plot the RGB image. 
The color vector is created by stacking the red, green, and blue bands of the
image. Instead of useing the median value, we could also use the data of a 
specific date.

In [ ]:
# define a mask for valid pixels (non-cloud)
def is_valid_pixel(data):
    # include only vegetated, not_vegitated, water, and snow
    return ((data > 3) & (data < 7)) | (data == 11)


dc["valid"] = is_valid_pixel(dc.scl)

In [ ]:
# compute the masked median
rgb_median = (
    dc[["red", "green", "blue"]]
    .where(dc.valid)
    .to_dataarray(dim="band")
    .median(dim="time")
    .astype(int)
)

# plot the median composite
title_rgb = (
    "RGB - Median Composite"
    + f"\n{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}"
)
rgb_median.plot.imshow(robust=True).axes.set_title(title_rgb)
plt.show()

To better understand the data, that is used to create the RGB image, we can
also plot the individual timestamps. This will give us a better understanding
of the data and the changes over time. Since the cloud coverage is not the same
for each image, the median composite image can be a better representation of 
the scene.

You can also remove the resampling of the data, to see the original data, which
have a lot of overlap. The resapmling can get rid of the overlap.

In [ ]:
# Resample to daily timestamps
daily = dc.red.resample(time='1D').max()

# Drop Images with no data
daily = daily.dropna('time', how='any')

# Plot the daily images of the red band
daily.plot.imshow(robust=True, col='time', col_wrap=3)

If you are interested in a specific date, you can also plot the data like 
explained in the following code snippet. 

In [ ]:
dc[['red', 'green', 'blue']].isel(time=3).to_array().plot.imshow(robust=True)

### False Color Image
In addition to the regular RGB Image, we can swap any of the bands from the
visible spectrum with any other bands. In this specific case the red band has
been changed to the near infrared band. This allows us to see vegetated areas
more clearly, since they now appear in a bright red color. This is due to the
fact that plants absorb regular red light while reflecting near infrared light.

In [ ]:
# compute a false color image
# near infrared instead of red
fc_median = (
    dc[['nir', 'green', 'blue']]
    .where(dc.valid)
    .to_dataarray(dim="band")
    .transpose(..., "band")
    .median(dim="time")
    .astype(int)
)

title_fc = (
    "False color - Median Composite"+
    f"\n{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}"
)
fc_median.plot.imshow(robust=True).axes.set_title(title_fc)
plt.show()

## Histogram
To get a better understanding of the data, we can plot a also plot the 
histogram of the data. This will give us an overview of the distribution of the
pixel values in the image. This can be useful to identify outliers or to manipulate
the data in a way that the contrast is increased in our visualization.

The `robust` parameter is used to exclude the 2% of the data with the highest
and lowest values, which can help to improve the visibility of the histogram.
It usually makes the visualization much better. If it does not work, you have
to resort to manual clipping of the data or other contarst enhancing methods.

In [ ]:
# Plot histograms of RGB bands
fig, ax = plt.subplots()
for band in ["red", "green", "blue"]:
    dc[band].median("time").plot.hist(
        bins=100, ax=ax, alpha=0.5, label=band, color=band
    )
ax.legend()
ax.set_title("Histograms of RGB bands")
plt.show()

### NDVI Image
To get an first impression of the data, we can calculate the NDVI (Normalized 
Difference Vegetation Index) and plot it. The NDVI is calculated by useing the
following formula.

$$
NDVI = \frac{NIR - Red}{NIR + Red}
$$

This gives us a good overview of the vegetation in the area. The values can
range from -1 to 1 where the following meanings are associated with these
values:

- -1 to 0 indicate dead plants or inanimate objects
- 0 to 0.33 are unhealthy plants
- 0.33 to 0.66 are moderatly healthy plants
- 0.66 to 1 are very healthy plants

In [ ]:
# Normalized Difference Vegetation Index (NDVI)
def normalized_difference(a, b):
    return (a - b * 1.0) / (a + b)


ndvi = normalized_difference(dc.nir, dc.red)
ndvi.median(dim="time").plot.imshow(
    cmap="RdYlGn", vmin=-1, vmax=1
).axes.set_title("NDVI")
plt.show()

## Exporting the Data
After we have visualized the data, we can export it to a GeoTIFF file. This
will allow us to use the data in other GIS software or to share it with others.

In [ ]:
# Select data that should be saved
# save = dc.sel(time="2024-05-09", method="nearest")

# Save the data to a GeoTIFF file
# save.rio.to_raster(
#     "sentinel_2_2024_05_09.tif", tiled=True, driver="GTiff", compress="LZW"
# )

In [ ]:
# Load the data again (for demonstration purposes)
# loaded_data = xr.open_dataset("sentinel_2_2024_05_09.tif", engine="rasterio")
# loaded_data

If you want to load multiple images, have a read in the
[xarray documentation](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html)
to see what else is possible. 